## Import the library

In [36]:
from librq import *
from bs4 import BeautifulSoup
import string
import re
import pandas as pd
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException

## Parameter Setting

In [37]:
page_apply = 'page-apply-btn'
page_cancel = 'page-cancel-btn'
du_Button = [page_apply, page_cancel]
Add = 'fa fa-plus '
class_Add = '.fa.fa-plus'
Copy = 'fa fa-copy '
class_Copy = '.fa.fa-copy'
Edit = 'fa fa-edit '
class_Edit = '.fa.fa-edit'
Radio = ['port_0', 'port_28','port_18', 'msti_0', 'selectedVid_0']


## Open the page of switch

In [38]:
driver = web_login()
webpage_ip = 'http://172.21.146.224/#!/'
time.sleep(1)
# html data
soup = BeautifulSoup(driver.page_source, 'html.parser')

## Get whole url in menu bar except the menu url

In [39]:
url_list = []
menu_html = soup.find(id = 'menu')
for link in menu_html.find_all('tr'):
    # print(link)
    #print("class:", link.get('class'))
    if link.get('id') is not None and 'hasChild' not in link.get('class'):
        url_list.append(link.get('id'))
# print("url_list:", url_list, '\n', len(url_list))

## Return specific tags or class 's id and some commn use methods 

In [40]:
def show_tag_ids(tag_list, tag_name):
    tag_id = []
    for i in range(len(tag_list)):
        for id_ele in str(tag_list[i]).split(" "):
            if re.match('^id=*', id_ele) != None:
                id_ele = id_ele.split("\"")[1]
                tag_id.append(id_ele)
                tag_id = sorted(set(tag_id), key=tag_id.index)
    return tag_id



def show_class_ids(url, class_soup, class_name):
    class_id = []
    for element in class_soup:
        class_name = ""
        for temp_str in element['class']:
            class_name+= temp_str
            if element['class'].index(temp_str) != -1:
                class_name+= " "
        # print(class_name)
        if Add in class_name or Copy in class_name or Edit in class_name:
            class_id.append(class_name)
            class_id = sorted(set(class_id), key=class_id.index)
            temp_type = class_name
            try:
                temp_id = element['id']
            except:
                try:
                    temp_id = "class=" 
                    temp_id += element['class']
                except:
                    temp_id = None
            temp_linktext = element.text
            if temp_linktext == "" :     
                try:
                    temp_linktext = "dn-label" + element['dn-label']
                except KeyError:
                    temp_linktext = ""
            element_dic['page_url'].append(url)
            element_dic['element_type'].append(temp_type)
            element_dic['element_id'].append(temp_id)
            element_dic['linktext'].append(temp_linktext)
    return class_id


def windows_tag_ids(windows_list, windows_name):
    windows_id = []
    for i in range(len(windows_list)):
        if re.match('.*disabled=\"disabled\"', str(windows_list[i])) == None:
            for id_ele in str(windows_list[i]).split(" "):
                if re.match('^id=*', id_ele) != None:
                    id_ele = id_ele.split("\"")[1]
                    windows_id.append(id_ele)
                    windows_id = sorted(set(windows_id), key=windows_id.index)
    return windows_id

def print_data(data, data_name):
    if data:
        # print(f'{data_name} length :  {len(data)}')
        print(f'{data_name} : ')
        print(data)
def get_elements_info(url, ele_type, type_soup):
    for element in type_soup:
        temp_type = ""
        if ele_type == 'input':
            temp_type = element['type']
        elif ele_type == 'input in add window':
            temp_type = element['type'] + " in add window"
        elif ele_type == 'input in edit window':
            temp_type = element['type'] + " in edit window"
            
        else:
            temp_type = ele_type
        
        try:
            temp_id = element['id']
        except:
            try:
                temp_id = "class=" 
                temp_id += element['class']
            except:
                temp_id = None
        try:
            temp_linktext = element.text
            if temp_linktext == "" :     
                try:
                    if element['dn-label'] != "":
                        temp_linktext = "dn-label:" + element['dn-label']    
                except KeyError:
                    temp_linktext = None
                    try:
                        if element['data-target'] != "":
                            temp_linktext = "data-target:" + element['data-target']
                    except:
                        temp_linktext = None
        except:
            temp_linktext = None
        # filter the element that must have the id or linktext can be record
        if temp_id is not None or temp_linktext is not None:
            element_dic['page_url'].append(url)
            element_dic['element_type'].append(temp_type)
            element_dic['element_id'].append(temp_id)
            element_dic['linktext'].append(temp_linktext)

## Define the function which can get the element from the page (modify the way to recognize the add , edir and copy button)

In [89]:
def get_elements_from_page(url, driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(1)
    # to get the content of the page html
    soup = soup.find(id = 'contentDiv')
    # print(soup)

    
    anchor_soup = soup.findAll("a")
    get_elements_info(url, "anchor", anchor_soup)
    
    # get the click element 
    click_soup = soup.findAll(True, attrs = {"class":"link-style ng-binding"})
    get_elements_info(url, "click_link", click_soup)
    
    input_soup = soup.findAll("input")
    get_elements_info(url, "input", input_soup)
    
    """
    text_soup = soup.findAll("input", type='text')
    get_elements_info(url, "text", click_soup)
    # text_list = show_tag_ids(url, text_soup, 'text_list')
    # print_data(text_list, 'text_list')

    radio_soup = soup.findAll("input", type='radio')
    get_elements_info(url, "radio_button", radio_soup)
    radio_list = show_tag_ids(radio_soup, 'radio_list')
    # print_data(radio_list, 'radio_list')
    
    checkbox_soup = soup.findAll("input", type='checkbox')
    get_elements_info(url, "checkbox", checkbox_soup)
    # checkbox_list = show_tag_ids(url, checkbox_soup, 'checkbox_list')
    # print_data(checkbox_list, 'checkbox_list')
    """

    textarea_soup = soup.findAll("textarea", id=True)
    get_elements_info(url, "textarea", textarea_soup)
    # textarea_list = show_tag_ids(url, textarea_soup, 'textarea_list')
    # print_data(textarea_list, 'textarea_list')

    button_soup = soup.findAll("button", id=True)
    get_elements_info(url, "button", button_soup)
    # button_list = show_tag_ids(url, button_soup, 'button_list')
    # print_data(button_list, 'button_list')

    select_soup = soup.findAll("select", id=True)
    get_elements_info(url, "drop-down list", select_soup)
    # select_list = show_tag_ids(url, select_soup, 'select_list')
    # print_data(select_list, 'select_list')

    tagi_soup = soup.findAll("i", class_=True)
    #print(tagi_soup)
    #get_elements_info(url, "input_style", tagi_soup)
    tagi_list = show_class_ids(url, tagi_soup, 'input_style')
    #print_data(tagi_list, 'tagi_list')
    # print(tagi_list)
    if Add in tagi_list:
        try:
            click_add_flag = True
            driver.find_element_by_css_selector(class_Add).click()
            time.sleep(2)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            try:
                soup = soup.find(True, attrs = {"class":'ngdialog-content'})
                
                """
                text_soup = soup.findAll("input", type='text')
                get_elements_info(url, "text in add window", text_soup)
                text_Add = windows_tag_ids(text_soup, 'text_Add')
                # text_Add = set(text_addwindow) - set(text_list)
                #print_data(text_Add, 'text_Add')

                radio_soup = soup.findAll("input", type='radio')
                get_elements_info(url, "radio in add window", radio_soup)
                radio_Add = windows_tag_ids(radio_soup, 'radio_Add')
                # radio_Add = set(radio_addwindow) - set(radio_list)
                #print_data(radio_Add, 'radio_Add')
                
                checkbox_soup = soup.findAll("input", type='checkbox')
                get_elements_info(url, "checkbox in add window", checkbox_soup)
                checkbox_Add = windows_tag_ids(checkbox_soup, 'checkbox_Add')
                # checkbox_Add = set(checkbox_addwindow) - set(checkbox_list)
                #print_data(checkbox_Add, 'checkbox_Add')
                """
                text_soup = soup.findAll("input")
                get_elements_info(url, "input in add window", text_soup)

                textarea_soup = soup.findAll("textarea", id=True)
                get_elements_info(url, "textarea in add window", textarea_soup)
                textarea_Add = windows_tag_ids(textarea_soup, 'textarea_Add')
                # textarea_Add = set(textarea_addwindow) - set(textarea_list)
                #print_data(textarea_Add, 'textarea_Add')

                button_soup = soup.findAll("button", id=True)
                get_elements_info(url, "button in add window", button_soup)
                button_Add = windows_tag_ids(button_soup, 'button_Add')
                #button_Add = set(button_addwindow) - set(button_list)
                #if (page_apply in button_list and page_cancel in button_list) and (
                        #page_apply in button_addwindow and page_cancel in button_addwindow):
                    #button_Add = set(button_Add) | set(du_Button)
                #print_data(button_Add, 'button_Add')

                select_soup = soup.findAll("select", id=True)
                get_elements_info(url, "drop-down list in add window", select_soup)
                select_Add = windows_tag_ids(select_soup, 'select_Add')
                #select_Add = set(select_addwindow) - set(select_list)
                #print_data(select_Add, 'select_Add')

                
            except:
                print("Do not find the ngdialog")          
        except:
            click_add_flag = False
            print(f'{url} => can not click Add_button')
            
    driver.refresh()
    time.sleep(2.5)
    if Edit in tagi_list :
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        entry_list = []
        # print(soup)
        soup = soup.find(id = "contentDiv")
        hide_tag = soup.find("div", class_ =  "ng-hide")
        get_tale_flag = False
        if hide_tag:
            print("hide_tag:", hide_tag['class'])
            if "alert" in hide_tag['class'] and "ngdialog-message" in hide_tag['class']and "ng-hide" in hide_tag['class'] :
                get_tale_flag = True
            elif "ng-hide" in hide_tag['class']:
                print("the page is hide by ng-hide")
        else:
            get_tale_flag = True
        if get_tale_flag :
            try: 
                soup = soup.find(class_ = "tableContent")
                table_soup = soup.find("table")
                tbody_soup = soup.findAll("tbody")
                #print(soup)
                if len(tbody_soup) > 1:
                    tbody_soup = tbody_soup[-1]
                    print("Do not have only one table!!")
                for table_ele in tbody_soup:
                    table_input = table_ele.findAll("input")
                    if table_input:
                        # print(table_input)
                        for t_input in table_input:
                            entry_list.append(t_input['id'])
                        # entry_list.append(table_input['id'])
                # print(table_soup)
                """

                for tr in tr_soup:
                    entry_list.append(tr.findAll("input"))

                # tr_soup = tbody_soup.findAll("tr")

                """
            except:
                print("Occur something error while finding entry list!!")
        
        else:
            print("Do not have to get the table!!")
        print(url, "Entry List:", entry_list)
        if entry_list:
            entry_no = entry_list[0]
            edit_btn = 'port-edit-btn' if 'port' in entry_no else 'tbtn-edit-btn'
            if 'msti' in entry_no:
                edit_btn = 'msti-edit-btn'
            elif 'selectedVid' in entry_no:
                edit_btn = 'mcast_vlan-edit-btn'
            # choose the entry to edit 
            num = len(entry_list)
            while(True):
                # print("enable_flag :", driver.find_element_by_id(edit_btn).get_attribute('disabled'))
                num -= 1
                if num == -1:
                    print("click all entry but we can not trigger the edit btn")
                    break
                selected_entry = entry_list[num]
                if not driver.find_element_by_id(selected_entry).is_selected():
                    try:
                        driver.find_element_by_id(selected_entry).click()
                        #print("click",selected_entry)
                        time.sleep(0.3)
                    except:
                        print("Can not click the entry")
                #print("state:", driver.find_element_by_id(edit_btn).get_attribute('disabled'))        
                if driver.find_element_by_id(edit_btn).get_attribute('disabled') == None:
                    break
                driver.find_element_by_id(selected_entry).click()
                
    
            try:    
                print("edit_btn", edit_btn)
                driver.find_element_by_id(edit_btn).click()
                time.sleep(1)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                try:
                    soup = soup.find(True, attrs = {"class":'ngdialog-content'})
                    
                    """
                    text_soup = soup.findAll("input", type='text')
                    get_elements_info(url, "text in edit window", text_soup)
                    text_Add = windows_tag_ids(text_soup, 'text_Add')
                    # text_Add = set(text_addwindow) - set(text_list)
                    print_data(text_Add, 'text_Add')

                    radio_soup = soup.findAll("input", type='radio')
                    get_elements_info(url, "radio in edit window", radio_soup)
                    radio_Add = windows_tag_ids(radio_soup, 'radio_Add')
                    # radio_Add = set(radio_addwindow) - set(radio_list)
                    print_data(radio_Add, 'radio_Add')
                    
                    checkbox_soup = soup.findAll("input", type='checkbox')
                    get_elements_info(url, "checkbox in edit window", checkbox_soup)
                    checkbox_Add = windows_tag_ids(checkbox_soup, 'checkbox_Edit')
                    # checkbox_Add = set(checkbox_addwindow) - set(checkbox_list)
                    print_data(checkbox_Add, 'checkbox_Add')
                    """
                    text_soup = soup.findAll("input")
                    get_elements_info(url, "input in edit window", text_soup)

                    textarea_soup = soup.findAll("textarea", id=True)
                    get_elements_info(url, "textarea in edit window", textarea_soup)
                    textarea_Add = windows_tag_ids(textarea_soup, 'textarea_Add')
                    # textarea_Add = set(textarea_addwindow) - set(textarea_list)
                    print_data(textarea_Add, 'textarea_Add')

                    button_soup = soup.findAll("button", id=True)
                    get_elements_info(url, "button in edit window", button_soup)
                    button_Add = windows_tag_ids(button_soup, 'button_Add')
                    #button_Add = set(button_addwindow) - set(button_list)
                    #if (page_apply in button_list and page_cancel in button_list) and (
                            #page_apply in button_addwindow and page_cancel in button_addwindow):
                        #button_Add = set(button_Add) | set(du_Button)
                    print_data(button_Add, 'button_Add')

                    select_soup = soup.findAll("select", id=True)
                    get_elements_info(url, "drop-down list in edit window", select_soup)
                    select_Add = windows_tag_ids(select_soup, 'select_Add')
                    #select_Add = set(select_addwindow) - set(select_list)
                    print_data(select_Add, 'select_Add')

                    
                except:
                    print("Do not find the ngdialog") 
            except ElementClickInterceptedException:
                print(f'{url} => can not click Edit_button after all.')
    # driver.refresh()
    time.sleep(1) 
                    
        
    """
        
        if radio_list:
            if Radio[0] in radio_list:
                radio_button = Radio[0]
            if Radio[1] in radio_list:
                radio_button = Radio[1]
            if Radio[2] in radio_list:
                radio_button = Radio[2]
            if Radio[3] in radio_list:
                radio_button = Radio[3]
            try:
                driver.find_element_by_id(radio_button).click()
                # click copy
                if Copy in tagi_list:
                    driver.find_element_by_css_selector(class_Copy).click()
                    time.sleep(1)
                    soup = BeautifulSoup(driver.page_source, 'html.parser')

                    text_soup = soup.findAll("input", type='text')
                    text_copywindow = windows_tag_ids(text_soup, 'text_Copy')
                    text_Copy = set(text_copywindow) - set(text_list)
                    print_data(text_Copy, 'text_Copy')

                    radio_soup = soup.findAll("input", type='radio')
                    radio_copywindow = windows_tag_ids(radio_soup, 'radio_Copy')
                    radio_Copy = set(radio_copywindow) - set(radio_list)
                    print_data(radio_Copy, 'radio_Copy')

                    textarea_soup = soup.findAll("textarea", id=True)
                    textarea_copywindow = windows_tag_ids(textarea_soup, 'textarea_Copy')
                    textarea_Copy = set(textarea_copywindow) - set(textarea_list)
                    print_data(textarea_Copy, 'textarea_Copy')

                    button_soup = soup.findAll("button", id=True)
                    button_copywindow = windows_tag_ids(button_soup, 'button_Copy')
                    button_Copy = set(button_copywindow) - set(button_list)
                    print_data(button_Copy, 'button_Copy')

                    select_soup = soup.findAll("select", id=True)
                    select_copywindow = windows_tag_ids(select_soup, 'select_Copy')
                    select_Copy = set(select_copywindow) - set(select_list)
                    print_data(select_Copy, 'select_Copy')

                    checkbox_soup = soup.findAll("input", type='checkbox')
                    checkbox_copywindow = windows_tag_ids(checkbox_soup, 'checkbox_Copy')
                    checkbox_Copy = set(checkbox_copywindow) - set(checkbox_list)
                    print_data(checkbox_Copy, 'checkbox_Copy')

                    driver.find_element_by_id('dialog-cancel').click()
                    time.sleep(1)

                # click edit
                if Edit in tagi_list:
                    driver.find_element_by_css_selector(class_Edit).click()
                    time.sleep(1)
                    soup = BeautifulSoup(driver.page_source, 'html.parser')

                    text_soup = soup.findAll("input", type='text')
                    text_editwindow = windows_tag_ids(text_soup, 'text_Edit')
                    text_Edit = set(text_editwindow) - set(text_copywindow) - set(text_list)
                    print_data(text_Edit, 'text_Edit')

                    radio_soup = soup.findAll("input", type='radio')
                    radio_editwindow = windows_tag_ids(radio_soup, 'radio_Edit')
                    radio_Edit = set(radio_editwindow) - set(radio_copywindow) - set(radio_list)
                    print_data(radio_Edit, 'radio_Edit')

                    textarea_soup = soup.findAll("textarea", id=True)
                    textarea_editwindow = windows_tag_ids(textarea_soup, 'textarea_Edit')
                    textarea_Edit = set(textarea_editwindow) - set(textarea_copywindow) - set(textarea_list)
                    print_data(textarea_Edit, 'textarea_Edit')

                    button_soup = soup.findAll("button", id=True)
                    button_editwindow = windows_tag_ids(button_soup, 'button_Edit')
                    button_Edit = set(button_editwindow) - set(button_copywindow) - set(button_list)
                    if ('page-cancel-btn' in button_list and 'page-apply-btn' in button_list) and (
                            'page-cancel-btn' in button_editwindow and 'page-apply-btn' in button_editwindow):
                        button_Edit = set(button_Edit) | set(du_Button)
                    print_data(button_Edit, 'button_Edit')

                    select_soup = soup.findAll("select", id=True)
                    select_editwindow = windows_tag_ids(select_soup, 'select_Edit')
                    select_Edit = set(select_editwindow) - set(select_copywindow) - set(select_list)
                    print_data(select_Edit, 'select_Edit')

                    checkbox_soup = soup.findAll("input", type='checkbox')
                    checkbox_editwindow = windows_tag_ids(checkbox_soup, 'checkbox_Edit')
                    checkbox_Edit = set(checkbox_editwindow) - set(checkbox_copywindow) - set(checkbox_list)
                    print_data(checkbox_Edit, 'checkbox_Edit')
            except:
                print(f'{url} => can not click radio_button')
    """

## Find all element in every page 

In [90]:
def find_whole_element():
    #for url_name in ["M_AdminCdpIntf"]:
    for url_name in url_list:
        t_s = time.time()
        page_url = webpage_ip + url_name
        driver.get(page_url)
        time.sleep(3)

        # print(page_url)

        get_elements_from_page(page_url, driver)
        print("Get element from ", page_url, " spend ", time.time() - t_s, "seconds")

In [91]:
time_start = time.time()
element_dic = {'page_url':[],'element_type':[],'element_id':[],'linktext':[]}
find_whole_element()
print("Spend ", time.time() - time_start, "to find whole element!!")

Get element from  http://172.21.146.224/#!/M_GettingStarted  spend  7.768664836883545 seconds
Get element from  http://172.21.146.224/#!/M_StatusSummary  spend  7.8679118156433105 seconds
Get element from  http://172.21.146.224/#!/M_StatusIntf  spend  7.765402555465698 seconds
Get element from  http://172.21.146.224/#!/M_StatusEtherlike  spend  7.740338563919067 seconds
Get element from  http://172.21.146.224/#!/M_StatusHwUtilz  spend  7.706826686859131 seconds
Get element from  http://172.21.146.224/#!/M_StatusHealth  spend  7.742983102798462 seconds
Get element from  http://172.21.146.224/#!/M_StatusSpanVLAN  spend  7.732736587524414 seconds
hide_tag: ['alert', 'ngdialog-message', 'ng-hide']
http://172.21.146.224/#!/M_StatusSpanSessDest Entry List: []
Get element from  http://172.21.146.224/#!/M_StatusSpanSessDest  spend  10.169306516647339 seconds
hide_tag: ['alert', 'ngdialog-message', 'ng-hide']
Do not have only one table!!
Occur something error while finding entry list!!
http://1

Get element from  http://172.21.146.224/#!/M_AdminTraceroute  spend  7.740523815155029 seconds
hide_tag: ['alert', 'ngdialog-message', 'ng-hide']
http://172.21.146.224/#!/M_PortSettings Entry List: ['port_0', 'port_1', 'port_2', 'port_3', 'port_4', 'port_5', 'port_6', 'port_7', 'port_8', 'port_9', 'port_10', 'port_11', 'port_12', 'port_13', 'port_14', 'port_15', 'port_16', 'port_17', 'port_18', 'port_19', 'port_20', 'port_21', 'port_22', 'port_23', 'port_24', 'port_25', 'port_26', 'port_27']
edit_btn port-edit-btn
button_Add : 
['page-apply-btn', 'page-cancel-btn']
select_Add : 
['port']
Get element from  http://172.21.146.224/#!/M_PortSettings  spend  9.751710891723633 seconds
Get element from  http://172.21.146.224/#!/M_PortErrRecoverySettings  spend  7.86096715927124 seconds
hide_tag: ['alert', 'ngdialog-message', 'ng-hide']
http://172.21.146.224/#!/M_PortLBD Entry List: ['port_0', 'port_1', 'port_2', 'port_3', 'port_4', 'port_5', 'port_6', 'port_7', 'port_8', 'port_9', 'port_10', '

Get element from  http://172.21.146.224/#!/M_StpMstpIntf  spend  7.951721429824829 seconds
Get element from  http://172.21.146.224/#!/M_MacStatic  spend  9.914674520492554 seconds
Get element from  http://172.21.146.224/#!/M_MacStaticFilter  spend  9.957323789596558 seconds
Get element from  http://172.21.146.224/#!/M_MacDynamic  spend  7.787987232208252 seconds
Get element from  http://172.21.146.224/#!/M_MacDynamicFilter  spend  7.8083014488220215 seconds
hide_tag: ['alert', 'ngdialog-message', 'ng-hide']
http://172.21.146.224/#!/M_MacReserved Entry List: []
Get element from  http://172.21.146.224/#!/M_MacReserved  spend  10.119075775146484 seconds
Get element from  http://172.21.146.224/#!/M_McastProp  spend  7.754132509231567 seconds
hide_tag: ['alert', 'ngdialog-message', 'ng-hide']
Do not have only one table!!
Occur something error while finding entry list!!
http://172.21.146.224/#!/M_McastStaticGroup Entry List: []
Get element from  http://172.21.146.224/#!/M_McastStaticGroup  s

Get element from  http://172.21.146.224/#!/M_SecDosSynProtect  spend  7.8817219734191895 seconds
Get element from  http://172.21.146.224/#!/M_SecDhcpProp  spend  7.761550664901733 seconds
Get element from  http://172.21.146.224/#!/M_SecDhcpVlan  spend  7.724182605743408 seconds
http://172.21.146.224/#!/M_SecDhcpSnoopingIntf Entry List: ['port_0', 'port_1', 'port_2', 'port_3', 'port_4', 'port_5', 'port_6', 'port_7', 'port_8', 'port_9', 'port_10', 'port_11', 'port_12', 'port_13', 'port_14', 'port_15', 'port_16', 'port_17', 'port_18', 'port_19', 'port_20', 'port_21', 'port_22', 'port_23', 'port_24', 'port_25', 'port_26', 'port_27']
edit_btn port-edit-btn
button_Add : 
['page-apply-btn', 'page-cancel-btn']
select_Add : 
['port']
Get element from  http://172.21.146.224/#!/M_SecDhcpSnoopingIntf  spend  9.631789445877075 seconds
Get element from  http://172.21.146.224/#!/M_SecDhcpSnoopingBinding  spend  7.798983097076416 seconds
Get element from  http://172.21.146.224/#!/M_SecDhcpStatistic  s

Get element from  http://172.21.146.224/#!/M_QosBasicIntf  spend  7.718795537948608 seconds
Get element from  http://172.21.146.224/#!/M_QosAdvGlobal  spend  7.750808954238892 seconds
http://172.21.146.224/#!/M_QosAdvClass => can not click Add_button
Get element from  http://172.21.146.224/#!/M_QosAdvClass  spend  17.833866119384766 seconds
http://172.21.146.224/#!/M_QosAdvAggPolicer => can not click Add_button
hide_tag: ['ng-hide']
the page is hide by ng-hide
Do not have to get the table!!
http://172.21.146.224/#!/M_QosAdvAggPolicer Entry List: []
Get element from  http://172.21.146.224/#!/M_QosAdvAggPolicer  spend  17.98155188560486 seconds
http://172.21.146.224/#!/M_QosAdvPolicy => can not click Add_button
Get element from  http://172.21.146.224/#!/M_QosAdvPolicy  spend  17.817706823349 seconds
http://172.21.146.224/#!/M_QosAdvPolicyClass => can not click Add_button
hide_tag: ['ng-hide']
the page is hide by ng-hide
Do not have to get the table!!
http://172.21.146.224/#!/M_QosAdvPoli

# Show the result and transform it into exel(push it to the cloud sheet)

In [92]:
# print(element_dic)
elements_df = pd.DataFrame(element_dic)
print(elements_df)
elements_df.to_excel('element_output.xlsx',sheet_name='auto_gen_elements')
def write_to_google_sheet(list_name):
    key='web_key.json'
    gc = pygsheets.authorize(service_account_file=key)
    sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1pK6ekzMWCHsKsEaD25OBNgRkUnc8FSOXLMSomIMnOZs/edit#gid=1604068312')
    ws = sh.worksheet_by_title(list_name)
    print(ws)
    ws.set_dataframe(elements_df, 'A1') 


                                       page_url                  element_type  \
0     http://172.21.146.224/#!/M_GettingStarted                        anchor   
1     http://172.21.146.224/#!/M_GettingStarted                        anchor   
2     http://172.21.146.224/#!/M_GettingStarted                    click_link   
3     http://172.21.146.224/#!/M_GettingStarted                    click_link   
4     http://172.21.146.224/#!/M_GettingStarted                    click_link   
...                                         ...                           ...   
3459        http://172.21.146.224/#!/M_SnmpTrap  drop-down list in add window   
3460      http://172.21.146.224/#!/M_SnmpTrapv3                        anchor   
3461      http://172.21.146.224/#!/M_SnmpTrapv3                      checkbox   
3462      http://172.21.146.224/#!/M_SnmpTrapv3                   fa fa-plus    
3463      http://172.21.146.224/#!/M_SnmpTrapv3                   fa fa-edit    

                   element_

# write to the cloud

In [93]:
write_to_google_sheet(list_name = "auto_gen_element")

<Worksheet 'auto_gen_element' index:9>


## Close driver

In [ ]:
driver.close()